In [9]:
import pandas as pd
import numpy as np 
from multiprocessing import Pool
from tqdm import tqdm
import os
import torch 

In [2]:
image_folder = "/home/user/DataCenter/HAR_data/house_images/"
img_paths = []
for i in tqdm(range(100)):
    for j in range(100):
        subfolder = os.path.join(image_folder, "%04d/%04d/"%(i,j))
        img_paths.extend([os.path.join(subfolder, f_) for f_ in os.listdir(subfolder)])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 119.86it/s]


In [3]:
img_df = pd.DataFrame.from_dict({"img_path": img_paths})
img_df["address"] = img_df["img_path"].apply(lambda x: x.split("/")[-1][:-11])
print(len(img_df))
img_df = img_df.drop_duplicates(["address"], keep="last")
print(len(img_df))
img_df.head(3)

421724
418962


,img_path,address
0,/home/user/DataCenter/HAR_data/house_images/00...,"2710 Sir Patrice Ln, Lewisville, TX 75056"
1,/home/user/DataCenter/HAR_data/house_images/00...,"2407 Dowd Ln, Austin, TX 78728"
2,/home/user/DataCenter/HAR_data/house_images/00...,"16381 Sun View Ln, Conroe, TX 77302"


In [7]:
df = pd.read_pickle("data/property_structured_v2.pkl")
df.head(2)

,address,LON,LAT,building_sqft,Lot Size,Year Built,Garage Number,Bedrooms,Baths,Maintenance Fee,...,finance_option_multiclass,Exterior Type_multiclass,Style_multiclass,price,elementary_school_star,middle_school_star,high_school_star,school_names,school_grades,school_org
0,"4513 Refugio Dr, Plano, TX 75024",-96.782005,33.102204,3395.0,429.0,2023.0,2.0,4.0,3.5,65.083333,...,None,"[brick, rock/stone]",[traditional],789990.0,5.0,5.0,NaN,"[Borchardt Elementary School Elementary, Fowle...","[KG - 05, 06 - 08]",[FRISCO ISD]
1,"1516 Bay Area Blvd P12, Houston, TX 77058",-95.114098,29.562197,684.0,31348.0,1977.0,NaN,1.0,1.0,272.000000,...,"[cash, conventional, fha, investor, va]","[brick, stucco]",[traditional],79900.0,4.0,NaN,5.0,"[Falcon Pass Elementary School Elementary, Spa...","[EE - 05, 06 - 08, 09 - 12]",[CLEAR CREEK ISD]


In [8]:
val_address_set = set(np.load("data/val_address.npy", allow_pickle=True))
cur_df = df[df["address"].apply(lambda x: x in val_address_set)]
cur_df = cur_df.join(img_df.set_index("address"), on="address", how="inner")

len(df), len(cur_df)

(219217, 43843)

In [23]:
cur_df.head(3)

,address,LON,LAT,building_sqft,Lot Size,Year Built,Garage Number,Bedrooms,Baths,Maintenance Fee,...,Exterior Type_multiclass,Style_multiclass,price,elementary_school_star,middle_school_star,high_school_star,school_names,school_grades,school_org,img_path
175374,"21715 Frogfruit Ct, Cypress, TX 77433,",-95.787578,29.942534,3203.0,645.0,2024.0,3.0,4.0,3.5,112.916667,...,[brick],[contemporary/modern],614900.0,3.0,NaN,3.0,"[Roberts Road Elementary School Elementary, Wa...","[EE - 05, 06 - 08, 09 - 12]",[WALLER ISD],/home/user/DataCenter/HAR_data/house_images/00...
175375,"21651 Wave Hollow Dr, Cypress, TX 77433,",-95.742624,29.894718,2404.0,481.0,2024.0,2.0,4.0,2.5,87.916667,...,[brick],[traditional],387990.0,4.0,4.0,3.0,"[Andre Elementary School Elementary, Rowe Midd...","[EE - 05, 06 - 08, 09 - 12]",[CYPRESS-FAIRBANKS ISD],/home/user/DataCenter/HAR_data/house_images/00...
175376,"10715 Spring White Dr, Cypress, TX 77433,",-95.813422,29.959213,3610.0,NaN,2024.0,3.0,4.0,4.0,112.916667,...,"[brick, stone, stucco]",[traditional],665990.0,3.0,NaN,3.0,"[Roberts Road Elementary School Elementary, Wa...","[EE - 05, 06 - 08, 09 - 12]",[WALLER ISD],/home/user/DataCenter/HAR_data/house_images/00...


In [ ]:
def read_and_stack_tensor(row):
    address, path = row 
    pt_paths = [os.path.join(path, pt_path) for pt_path in os.listdir(path) if pt_path.endswith(".pt")]
    pt_list = np.array([torch.load(pt_path).numpy() for pt_path in pt_paths])
    if len(pt_list) == 0:
        return 0
    else: 
        np.save(os.path.join("data/image_emb/", address + ".npy"), pt_list)
        return len(pt_list)

with Pool(10) as p:
    res = p.map(read_and_stack_tensor, cur_df[["address", "img_path"]].values)
cur_df["bag_length"] = res 


In [31]:
cur_df = cur_df[cur_df["bag_length"] > 0]
cur_df["bag_length"].max()

56

In [33]:
cur_df["array_path"] = cur_df["address"].apply(lambda x: os.path.join("datasets/image_emb/", x + ".npy"))
cur_df["target"] = cur_df["price"]

In [36]:
cur_df[["array_path", "bag_length", "target"]].to_csv("data/val_image_emb.csv", index=False)